# Day Eight

## Task

Our [task](https://adventofcode.com/2017/day/8) is to execute a bunch of instructions over a register and determine some value of the register after all intructions have been executed.

### Part One

First, we are given a set of instructions and asked to find the largest value in the register after all instructions have run.

Let's start by defining our register. I'm going to use a dictionary because the instructions suggest I will need a way to look up by position in the register.

In [1]:
registers = {
    'a': 0,
    'b': 0,
    'c': 0,
}

An instruction comes in the form `b inc 5 if a > 1` so let's translate this into a function that can be executed on register.

In [2]:
def read_operation(parts):
    return lambda x: x + int(parts[2])


def read_condition(parts):
    return lambda x: x > int(parts[6])

    
def read_instruction(string):
    parts = string.split(' ')
    return read_operation(parts), read_condition(parts)

Now when we call `read_instruction` on an instruction string, we will be returned with two functions; the first is the register operation and the second is a condition which determines whether we should execute the operation or not. For example:

In [3]:
operation, condition = read_instruction('b inc 5 if a > 1')

print(f"Operation: {operation(registers['b'])}, condition: {condition(registers['a'])}")

Operation: 5, condition: False


Now we extend `read_operation` and `read_condition` to handle the various values that might come as instructions.

In [4]:
operations = {
    'inc': lambda x: lambda y: y + x,
    'dec': lambda x: lambda y: y - x,
}


conditions = {
    '>': lambda x: lambda y: y > int(x),
    '>=': lambda x: lambda y: y >= int(x),
    '<': lambda x: lambda y: y < int(x),
    '<=': lambda x: lambda y: y <= int(x),
    '==': lambda x: lambda y: y == int(x),
    '!=': lambda x: lambda y: y != int(x),
}


def read_operation(parts):
    operation_type = parts[1]
    operation_value = int(parts[2])
    
    try:
        return operations[operation_type](operation_value)
    except KeyError:
        raise TypeError(f'Operation not recognised: {parts[1]}')


def read_condition(parts):
    condition_type = parts[5]
    condition_value = int(parts[6])
    
    try:
        return conditions[condition_type](condition_value)
    except KeyError:
        raise TypeError(f'Operation not recognised: {parts[5]}')

    
def read_instruction(string):
    parts = string.split(' ')
    return read_operation(parts), read_condition(parts)

Now we can run the set of instructions against our registers and determine the maximum value as follows.

In [5]:
instructions = [
    'b inc 5 if a > 1',
    'a inc 1 if b < 5',
    'c dec -10 if a >= 1',
    'c inc -20 if c == 10',
]


def run_instructions(registers, instructions):
    for instruction in instructions:
        operation_register = instruction.split(' ')[0]
        condition_register = instruction.split(' ')[4]
    
        operation, condition = read_instruction(instruction)
        if condition(registers[condition_register]):
            registers[operation_register] = operation(registers[operation_register])
            
            
run_instructions(registers, instructions)
print(f'The maximum value is: {max(registers.values())}')

The maximum value is: 1


Finally, we can do this against the puzzle input as follows.

In [6]:
from collections import defaultdict


with open('../../data/day8.txt') as f:
    data = f.read()
    

registers = defaultdict(lambda: 0)
instructions = [x for x in data.split('\n') if x]


run_instructions(registers, instructions)
print(f'The maximum value is: {max(registers.values())}')

The maximum value is: 5221


### Part Two

For part two, we need to keep track of the largest value ever held. The method that comes to mind immediately is to simply track it in `run_instructions`.

In [7]:
def run_instructions(registers, instructions):
    max_value = 0
    
    for instruction in instructions:
        operation_register = instruction.split(' ')[0]
        condition_register = instruction.split(' ')[4]
    
        operation, condition = read_instruction(instruction)
        if condition(registers[condition_register]):
            registers[operation_register] = operation(registers[operation_register])
            if registers[operation_register] > max_value:
                max_value = registers[operation_register]
                
    return max_value
                

registers = defaultdict(lambda: 0)
instructions = [x for x in data.split('\n') if x]


max_value = run_instructions(registers, instructions)
print(f'The maximum value currently is: {max(registers.values())}')
print(f'The maximum value ever written was: {max_value}')

The maximum value currently is: 5221
The maximum value ever written was: 7491
